# Markov models: What could go wrong?
<img alt="FU BERLIN LOGO" style="border-width:0" src="https://www.fu-berlin.de/assets/default2/fu-logo-1x-2e301182ca3c9a45ae34adf6dca8b6ba.png" title='FU BERLIN.' align="right"/>

```
# Tim Hempel
# PyEMMA Winterschool 2020
# Noe Group, FU Berlin
```

In [ ]:
import numpy as np
import pyemma
import mdshare
import matplotlib.pyplot as plt

In [ ]:
plt.matplotlib.rcParams.update({'font.size': 16})

## well-sampled double-well potential

In [ ]:
file = mdshare.fetch('hmm-doublewell-2d-100k.npz', working_directory='data')
with np.load(file) as fh:
    trjs = [fh['trajectory'][:, 1]]


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].hist(np.concatenate(trjs), bins=50, range=(-3, 3), alpha=1, density=True);
ax[0].plot(trjs[0][:200], np.linspace(0, 1, 200), color='k', alpha=.7)

cl = pyemma.coordinates.cluster_regspace(trjs, dmin=.05)
its = pyemma.msm.its(cl.dtrajs, lags=[i + 1 for i in range(10)])
pyemma.plots.plot_implied_timescales(its, marker='o', ax=ax[1])

fig.tight_layout()

In [ ]:
def plot_dw(trjs, hist=False, cl=None, ax=None, no_timeseries=False):
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    if hist:
        ax.hist(trjs[0], bins=30, range=(-4, 4), alpha=.5, density=True);
        ax.hist(trjs[1], bins=30, range=(-4, 4), alpha=.5, density=True);
    if cl is None and not no_timeseries:
        ax.plot(trjs[0], np.linspace(0, 1, trjs[0].shape[0]), color='C0')
        ax.plot(trjs[1], np.linspace(0, 1, trjs[1].shape[0]), color='C1')
    elif not no_timeseries:
        ax.plot(cl.clustercenters[cl.dtrajs[0], 0], np.linspace(0, 1, cl.dtrajs[0].shape[0]), color='C0')
        ax.plot(cl.clustercenters[cl.dtrajs[1], 0], np.linspace(0, 1, cl.dtrajs[1].shape[0]), color='C1')
        
    ax.set_xlim(-2, 2)
    return ax

## different kinds of almost well-sampled double well potentials
### 1. irreversibly connected

In [ ]:
file = mdshare.fetch('doublewell_oneway.npy', working_directory='data')
trjs = [trj for trj in np.load(file)]

In [ ]:
plot_dw(trjs, hist=True);

In [ ]:
cl = pyemma.coordinates.cluster_regspace(trjs, dmin=.7)
#cl = pyemma.coordinates.cluster_regspace(trjs, dmin=.1)
print(cl.n_clusters)

In [ ]:
plot_dw(trjs, cl=cl, hist=True);

In [ ]:
its = pyemma.msm.its(cl.dtrajs, lags=[1, 10, 100, 200, 300, 500, 800, 1000])
pyemma.plots.plot_implied_timescales(its, marker='o')
plt.ylim(1e1, 3e5);

In [ ]:
m = its.models[3]

ax = plot_dw(trjs, hist=True)
ax.set_yticks([])
tx = ax.twinx()
tx.plot(cl.clustercenters[m.active_set, 0], m.eigenvectors_right()[:, 1], 'ko:', label='first eigvec')
tx.set_ylabel('eigenvector')
ax.figure.legend()
ax.set_xlim(-2, 2)

In [ ]:
m.active_set

### 2. disconnected with crossovers

In [ ]:
file = mdshare.fetch('doublewell_disconnected.npy', working_directory='data')
trjs = [trj for trj in np.load(file)]

In [ ]:
plot_dw(trjs, hist=True);

In [ ]:
cl = pyemma.coordinates.cluster_regspace(trjs, dmin=.7)
#cl = pyemma.coordinates.cluster_regspace(trjs, dmin=.1)
print(cl.n_clusters)

In [ ]:
plot_dw(trjs, cl=cl, hist=True);

In [ ]:
its = pyemma.msm.its(cl.dtrajs, lags=[1, 10, 100, 200, 300, 500, 800, 1000])
pyemma.plots.plot_implied_timescales(its, marker='o')
plt.ylim(1e1, 3e5);

In [ ]:
m = pyemma.msm.estimate_markov_model(cl.dtrajs, lag=200)
m.pcca(2)

fig, ax = plt.subplots(1, 2, figsize=(10, 4))
index_order = np.argsort(cl.clustercenters[:, 0])
for n, metastable_distribution in enumerate(m.metastable_distributions):
    ax[0].step(cl.clustercenters[index_order, 0], metastable_distribution[index_order], ':', 
               label='metastable distr state {}'.format(n), where='mid')
ax[0].set_title('metastable distributions')
for _ax in ax:
    tx = _ax.twinx()
    tx.set_yticklabels([]); tx.set_yticks([])
    plot_dw(trjs, hist=True, ax=tx, no_timeseries=True)
    
ax[1].step(cl.clustercenters[index_order, 0], -np.log(m.pi[index_order]), 'k--', 
           label='potential from MSM $\pi$', where='mid')
fig.legend(loc='center right')
ax[1].set_title('stat dist');

## some "real world data"

In [ ]:
import mdshare

In [ ]:
pdb = mdshare.fetch('alanine-dipeptide-nowater.pdb', working_directory='data')
files = mdshare.fetch('alanine-dipeptide-*-250ns-nowater.xtc', working_directory='data')
feat = pyemma.coordinates.featurizer(pdb)

feat.add_all()
data = pyemma.coordinates.load(files, features=feat)

### ill-conducted TICA analysis

In [ ]:
tica = pyemma.coordinates.tica(data, lag=data[0].shape[0] - 100, dim=2)
tica_output = tica.get_output()

pyemma.plots.plot_free_energy(*np.concatenate(tica_output).T, legacy=False);

### discretization and MSM estimation

In [ ]:
cluster = pyemma.coordinates.cluster_kmeans(tica_output, k=200, max_iter=30, stride=100)

In [ ]:
its = pyemma.msm.its(cluster.dtrajs, lags=[1, 5, 10, 20, 30, 50])
pyemma.plots.plot_implied_timescales(its, marker='o', units='ps', nits=3);

##### -> "converged"
### coarse graining into two states

In [ ]:
msm = pyemma.msm.estimate_markov_model(cluster.dtrajs, 20)

nstates = 2
msm.pcca(nstates);

In [ ]:
stride = 10
metastable_trajs_strided = [msm.metastable_assignments[dtrj[::stride]] for dtrj in cluster.dtrajs]
tica_output_strided = [i[::stride] for i in tica_output]
_, _, misc = pyemma.plots.plot_state_map(*np.concatenate(tica_output_strided).T, 
                                         np.concatenate(metastable_trajs_strided));
misc['cbar'].set_ticklabels(range(1, nstates + 1))  # set state numbers 1 ... nstates

In [ ]:
pyemma.plots.plot_cktest(msm.cktest(nstates), units='ps');

# 👍 AWESOME !


What could be wrong?


### let's have a look at the trajectories as assigned to PCCA coarse states

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 6), sharey=True, sharex=True)
ax_yticks_labels = []
for n, pcca_traj in enumerate(metastable_trajs_strided):
    ax.plot(range(len(pcca_traj)), msm.n_metastable * n + pcca_traj, color='k', linewidth=0.3)
    ax.scatter(range(len(pcca_traj)), msm.n_metastable * n + pcca_traj, c=pcca_traj, s=0.1)
    ax_yticks_labels.append(((msm.n_metastable * (2 * n + 1) - 1) / 2, n + 1))
ax.set_yticks([l[0] for l in ax_yticks_labels])
ax.set_yticklabels([str(l[1]) for l in ax_yticks_labels])
ax.set_ylabel('Trajectory #')
ax.set_xlabel('time / {} ps'.format(stride))
fig.tight_layout()

### histogram view doesn't show connectedness nor metastability
### Take-away: Stay as close as possible to the trajectories

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 6), sharex=True, sharey='row')

for n, trj in enumerate(tica_output):
    for dim, traj1d in enumerate(trj.T):
        axes[dim, n].plot(traj1d[::stride], linewidth=.5)
for ax in axes[1]:
    ax.set_xlabel('time / {} ps'.format(stride))
for dim, ax in enumerate(axes[:, 0]):
    ax.set_ylabel('IC {}'.format(dim + 1))
for n, ax in enumerate(axes[0]):
    ax.set_title('Trajectory # {}'.format(n + 1))
fig.tight_layout()

# Thanks for your attention.